We want to compute the number of points on the curve, i.e., the order of the abelian group $|E(\mathbb{F}_p)|$ for a finite group $\mathbb{F}_p = \mathbb{Z}/p\mathbb{Z}$. Note that we should avoid the inefficient brute-force search of all $p^2$ pairs $(x, y)$. Instead, we iterate through each possible value of $x \in \mathbb{F}_p$ and calculate the number of corresponding $y$ values that satisfy the curve's equation. The total number of points is the sum of these affine solutions plus the single point at infinity $O = [0, 1, 0]$.

The Weierstrass equation is
\begin{equation}
    y^2 + a_1xy + a_3y = x^3 + a_2x^2 + a_4x + a_6,
\end{equation}
which can be rewritten as a quadratic equation in $y$
\begin{equation}
    y^2 + (a_1x + a_3)y - (x^3 + a_2x^2 + a_4x + a_6) = 0
\end{equation}
The number of solutions for $y$ depends on the prime $p$.

*   Case 1: $p > 2$.
We can complete the square for the $y$ terms,
\begin{equation}
    (y + (a_1x + a_3) 2^{-1})^2 = x^3 + a_2x^2 + a_4x + a_6 + ((a_1x + a_3)2^{-1})^2.
\end{equation}
We are solving an equation of the form $Y^2 = v$. The number of solutions for $Y$ (and thus for $y$) is determined by whether $v$ is a quadratic residue modulo $p$. This can be found using the Legendre symbol $\left(\frac{v}{p}\right)$ where the number of solutions is given by $1 + \left(\frac{v}{p}\right)$. We sum this quantity for each $x$ from $0$ to $p-1$.

*   Case 2: $p = 2$. In $\mathbb{F}_2$, division by $2$ is undefined, so we handle this case separately. The equation becomes
\begin{equation}
y^2 + a_1xy + a_3y = x^3 + a_2x^2 + a_4x + a_6.
\end{equation}
Since $y^2 = y$ in $\mathbb{F}_2$, we have
\begin{equation}
    y(1 + a_1x + a_3) = x^3 + a_2x^2 + a_4x + a_6.
\end{equation}
For $x = 0$ and $x = 1$, we solve this linear equation for y,
    *   If $1 + a_1x + a_3 \equiv 1 \mod 2$, then there is exactly one solution for $y$.
    *   If $1 + a_1x + a_3 \equiv 0 \mod 2$, then the LHS is $0$. If the RHS is also $0$, then we get two solutions $y=0$ and $y=1$. If the RHS is $1$, then the we get no solutions.

The algorithm iterates through all $p$ possible values for $x$. For $p > 2$, inside the loop, the main cost is calculating the Legendre symbol, which is done via modular exponentiation in $O(\log p)$ time. For $p = 2$, the loop runs a constant two times. The initial discriminant calculation is constant with respect to $p$. Therefore, the total time complexity of the method is $O(p \log p)$, which is significantly more efficient than the naive $O(p^2)$ approach.

We also output the trace $t_p := p + 1 - |E(\mathbb{F}_p)|$. Hasse's theorem says that $|t_p| \leq 2\sqrt{p}$ for any elliptic curve $E$ over $\mathbb{F}_p$.

Given an elliptic curve $E$ over the rational numbers $\mathbb{Q}$, which we will assume is defined by integers $a_1, a_2, a_3, a_4, a_6$, we get a family of cubic curves over $\mathbb{F}_p$, as the prime number $p$ varies, by reducing the coefficients $a_i$ modulo $p$. The resulting cubic curve is actually an elliptic curve over $F_p$ if and only if
\begin{equation}
    \Delta(E) \not\equiv 0 \mod p
\end{equation}
In that case, we say that $E$ has good reduction at $p$, otherwise that $E$ has bad reduction at $p$. Given a rational elliptic curve want to compute the discriminant and its the prime factorisation, which will allow us to compute the order and trace of $E(\mathbb{F}_p)$ for all primes $p \in [p_1, p_2]$ for which $E$ has a good reduction.

In [30]:
import math

class EllipticCurve:
    '''
    A class for elliptic curves over integers (Z) or finite fields (Fp).

    Represents curves in generalized Weierstrass form:
    y^2 + a1*x*y + a3*y = x^3 + a2*x^2 + a4*x + a6
    '''

    def __init__(self, coeffs, p=None):
        '''
        Initializes the elliptic curve.
        Args:
            coeffs: A list of 6 integer coefficients [a1, a2, a3, a4, a6].
            p: A prime number for a finite field Fp.
                               If None, the curve is treated as being over the integers Z.
        '''
        if p and not self._is_prime(p):
            raise ValueError("p must be a prime number.")

        self.p = p
        self.coeffs = coeffs

        if p:
            self.a1, self.a2, self.a3, self.a4, self.a6 = [c % p for c in coeffs]
        else:
            self.a1, self.a2, self.a3, self.a4, self.a6 = coeffs

        self._calculate_invariants()

    def _calculate_invariants(self):
        '''
        Internal helper to calculate b-invariants and the discriminant.
        '''
        a1, a2, a3, a4, a6 = self.a1, self.a2, self.a3, self.a4, self.a6

        self.b2 = a1**2 + 4 * a2
        self.b4 = a1 * a3 + 2 * a4
        self.b6 = a3**2 + 4 * a6
        self.b8 = a1**2 * a6 - a1 * a3 * a4 + 4 * a2 * a6 + a2 * a3**2 - a4**2
        self.delta = -self.b2**2 * self.b8 - 8 * self.b4**3 - 27 * self.b6**2 + 9 * self.b2 * self.b4 * self.b6

        if self.p:
            self.b2 %= self.p
            self.b4 %= self.p
            self.b6 %= self.p
            self.b8 %= self.p
            self.delta %= self.p

    def __str__(self):
        '''
        String representation of the curve.
        '''
        field_str = f"(mod {self.p})" if self.p else "over Z"
        return (f"y^2 + {self.a1}xy + {self.a3}y = "
                f"x^3 + {self.a2}x^2 + {self.a4}x + {self.a6} {field_str}")

    def is_singular(self):
        '''
        Checks if the curve is singular (discriminant is zero).
        '''
        return self.delta == 0

    def is_on_curve(self, P):
        '''
        Checks if a point P = (x, y) is on the curve. The point at infinity is always on the curve.
        '''
        if P is None:
            return True # Point at infinity
        if not self.p:
             raise TypeError("Point checking is only implemented for curves over Fp.")

        x, y = P
        lhs = (y**2 + self.a1*x*y + self.a3*y) % self.p
        rhs = (x**3 + self.a2*x**2 + self.a4*x + self.a6) % self.p
        return lhs == rhs

    # --- Methods for Curves over Finite Fields ---

    def _check_field(self, operation):
        '''
        Helper to ensure methods are called on an Fp curve.
        '''
        if not self.p:
            raise TypeError(f"'{operation}' is only defined for curves over a finite field Fp.")
        if self.is_singular():
            raise ValueError(f"Cannot perform group operations on a singular curve.")

    def negate(self, P):
        '''
        Computes -P, the inverse of point P in the elliptic curve group.
        '''
        self._check_field("Point negation")
        if P is None:
            return None # -O = O
        x, y = P
        return (x, (-y - self.a1*x - self.a3) % self.p)

    def add(self, P, Q):
        '''
        Adds two points P and Q on the curve using the group law.
        '''
        self._check_field("Point addition")
        if P is None: return Q
        if Q is None: return P

        x1, y1 = P
        x2, y2 = Q

        # Case P = -Q
        if x1 == x2 and (y1 + y2 + self.a1*x1 + self.a3) % self.p == 0:
            return None

        # Calculate slope
        if x1 == x2: # Point Doubling
            num = (3*x1**2 + 2*self.a2*x1 + self.a4 - self.a1*y1) % self.p
            den = (2*y1 + self.a1*x1 + self.a3) % self.p
        else: # Point Addition
            num = (y2 - y1) % self.p
            den = (x2 - x1) % self.p

        inv_den = pow(den, self.p - 2, self.p)
        lam = (num * inv_den) % self.p

        # Calculate new coordinates
        x3 = (lam**2 + self.a1*lam - self.a2 - x1 - x2) % self.p
        y3 = (-(lam + self.a1)*x3 - (y1 - lam*x1) - self.a3) % self.p

        return (x3, y3)

    def multiply(self, k, P):
        '''
        Computes scalar multiplication k * P using the double-and-add algorithm.
        '''
        self._check_field("Scalar multiplication")
        if k == 0: return None
        if k < 0:
            k = -k
            P = self.negate(P)

        result = None
        current = P
        while k > 0:
            if k % 2 == 1:
                result = self.add(result, current)
            current = self.add(current, current)
            k //= 2
        return result

    def order(self):
        '''
        Computes the order |E(Fp)| and trace t_p for the curve.
        '''
        self._check_field("Order calculation")

        order_val = 1
        if self.p == 2:
            for x in range(2):
                rhs = (x**3 + self.a2*x**2 + self.a4*x + self.a6) % 2
                lhs_factor = (1 + self.a1*x + self.a3) % 2
                if lhs_factor == 1: order_val += 1
                elif rhs == 0: order_val += 2
        else:
            for x in range(self.p):
                rhs = (x**3 + self.a2*x**2 + self.a4*x + self.a6) % self.p
                k = (self.a1 * x + self.a3) % self.p
                v = (rhs + (k * pow(2, self.p-2, self.p))**2) % self.p
                legendre = pow(v, (self.p - 1) // 2, self.p)
                if legendre == self.p - 1: legendre = -1
                order_val += (1 + legendre)

        return order_val, self.p + 1 - order_val

    # --- Methods for Curves over Integers ---

    def analyze_reduction_range(self, p_start, p_end):
        '''
        Analyzes the curve by reducing it over primes in the range [p_start, p_end).
        '''
        if self.p:
            raise TypeError("Reduction analysis can only be run on a curve defined over Z.")

        print(f"--- Analysis for: {self} ---")
        print(f"Discriminant Δ = {self.delta}")
        factors = self._prime_factorize(self.delta)
        factor_str = " * ".join([f"{p}^{e}" for p, e in factors.items()])
        print(f"Prime Factorization of |Δ|: {factor_str}\n")
        bad_primes = set(factors.keys())

        print(f"{'p':>5} | {'|E(Fp)|':>8} | {'t_p':>8}")
        print("-" * 29)
        for p_val in self._get_primes(p_start, p_end):
            if p_val in bad_primes:
                print(f"{p_val:>5} | {'***':>8} | {'***':>8}")
            else:
                reduced_curve = EllipticCurve(self.coeffs, p=p_val)
                order_val, t_p = reduced_curve.order()
                print(f"{p_val:>5} | {order_val:>8} | {t_p:>8}")
        print("\n")

    @staticmethod
    def _is_prime(n):
        if n < 2: return False
        for i in range(2, int(math.sqrt(n)) + 1):
            if n % i == 0: return False
        return True

    @staticmethod
    def _prime_factorize(n):
        n = abs(n)
        factors = {}
        d = 2
        while d * d <= n:
            while (n % d) == 0:
                factors[d] = factors.get(d, 0) + 1
                n //= d
            d += 1
        if n > 1: factors[n] = factors.get(n, 0) + 1
        return factors

    @staticmethod
    def _get_primes(start, end):
        for num in range(max(2, start), end):
            if EllipticCurve._is_prime(num):
                yield num

print("Reduction Analysis of a Curve over Z")
curve_a = EllipticCurve([0, 7, 0, 2, 0])
curve_b = EllipticCurve([1, 1, 1, -5, -7])
curve_c = EllipticCurve([0, -14, 0, 41, 0])
curve_a.analyze_reduction_range(0, 200)
curve_b.analyze_reduction_range(0, 200)
curve_c.analyze_reduction_range(0, 200)


print("Operations on a Curve over Fp")
p = 97
curve_fp = EllipticCurve(coeffs=[0, 0, 0, 2, 3], p=p)
print(f"Working with curve: {curve_fp}")
print(f"Is curve singular? {curve_fp.is_singular()}")
order, t_p = curve_fp.order()
print(f"The order |E(Fp)| is {order}, and the trace t_p is {t_p}.\n")
P = (3, 6)
Q = (10, 33)
print(f"Is P = {P} on the curve? {curve_fp.is_on_curve(P)}")
print(f"Is Q = {Q} on the curve? {curve_fp.is_on_curve(Q)}")
neg_P = curve_fp.negate(P)
P_plus_Q = curve_fp.add(P, Q)
two_P = curve_fp.add(P, P) # Same as multiply(2, P)
five_P = curve_fp.multiply(5, P)
print(f"-P = {neg_P}")
print(f"P + Q = {P_plus_Q}")
print(f"2 * P = {two_P}")
print(f"5 * P = {five_P}")
print(f"Is {order} * P = O (the point at infinity)? {curve_fp.multiply(order, P) is None}")

Reduction Analysis of a Curve over Z
--- Analysis for: y^2 + 0xy + 0y = x^3 + 7x^2 + 2x + 0 over Z ---
Discriminant Δ = 2624
Prime Factorization of |Δ|: 2^6 * 41^1

    p |  |E(Fp)| |      t_p
-----------------------------
    2 |      *** |      ***
    3 |        6 |       -2
    5 |        8 |       -2
    7 |        8 |        0
   11 |        6 |        6
   13 |       18 |       -4
   17 |       12 |        6
   19 |       22 |       -2
   23 |       24 |        0
   29 |       38 |       -8
   31 |       24 |        8
   37 |       36 |        2
   41 |      *** |      ***
   43 |       52 |       -8
   47 |       48 |        0
   53 |       66 |      -12
   59 |       56 |        4
   61 |       68 |       -6
   67 |       78 |      -10
   71 |       84 |      -12
   73 |       64 |       10
   79 |       72 |        8
   83 |       92 |       -8
   89 |       96 |       -6
   97 |      100 |       -2
  101 |      106 |       -4
  103 |      120 |      -16
  107 |      108 |   